<a href="https://colab.research.google.com/github/younesabdolmalaky/simple-chat-bot/blob/main/ChatbotAttentionNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import pandas as pd

In [54]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [55]:
def preprocess_senetence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)
    w = w.rstrip().strip()
    w = '<strat> ' + w + ' <end>'
    return w

In [56]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [57]:
def tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [58]:
def load_dataset(path, num_examples=None):
    df = pd.read_csv('/content/dialogs.txt',sep='\t' , names = ['qus' , 'ans'])
    inp_lang = df['qus']
    targ_lang = df['ans']
    inp_lang = inp_lang.apply(preprocess_senetence)
    targ_lang = targ_lang.apply(preprocess_senetence)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [59]:
path_to_file = '/content/dialogs.txt'

In [60]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)

In [61]:
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [63]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [64]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

In [65]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [66]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [67]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [68]:
encoder

In [69]:
simple_hidden = encoder.initilize_hidden_state()

In [70]:
example_input_batch, example_target_batch = next(iter(dataset))

In [71]:
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [72]:
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [73]:
attention_layer = Attention(10)


In [31]:
attention_result, attention_weights = attention_layer(simple_hidden, simple_output)

In [32]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [33]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [35]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [36]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [37]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [38]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [39]:
import time 

In [40]:
EPOCH = 100
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        start = time.time()
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        end = time.time()
        print('time: ', end - start)
        print('Epoch: ', epoch)
        print('Loss: ', batch_loss.numpy())
    checkpoint.save(file_prefix='test1')

time:  9.217735767364502
Epoch:  0
Loss:  3.1644964
time:  1.0533947944641113
Epoch:  0
Loss:  2.6362174
time:  1.0678856372833252
Epoch:  0
Loss:  2.772146
time:  1.0532371997833252
Epoch:  0
Loss:  2.6294649
time:  1.0621612071990967
Epoch:  0
Loss:  2.8173275
time:  0.8665311336517334
Epoch:  0
Loss:  2.2959116
time:  1.0470256805419922
Epoch:  0
Loss:  2.315863
time:  0.8492817878723145
Epoch:  0
Loss:  2.2259402
time:  1.121849775314331
Epoch:  0
Loss:  2.0730305
time:  1.4735946655273438
Epoch:  0
Loss:  2.1963422
time:  1.0733022689819336
Epoch:  0
Loss:  2.1616414
time:  0.662851095199585
Epoch:  0
Loss:  2.108451
time:  0.8402297496795654
Epoch:  0
Loss:  1.9200655
time:  1.056610107421875
Epoch:  0
Loss:  2.1024466
time:  1.0729787349700928
Epoch:  0
Loss:  2.0974994
time:  1.087644100189209
Epoch:  0
Loss:  1.953272
time:  1.0589675903320312
Epoch:  0
Loss:  1.8504878
time:  1.1586599349975586
Epoch:  0
Loss:  2.0269992
time:  1.1953589916229248
Epoch:  0
Loss:  2.1483955
ti

In [41]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [42]:
checkpoint.restore(tf.train.latest_checkpoint(''))

In [47]:
!pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import random
import pickle


while True:
    print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
    inp = input()
    if inp.lower() == "quit":
        break
    result = evaluate(inp)    
    print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL ,result[0])

      # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)

User: hi
ChatBot: yes , but i enjoy the street , but i enjoy the street , but i enjoy the street , but i enjoy the 
User: quit
Start messaging with the bot (type quit to stop)!
